# Démonstration : Modes AES (ECB vs CBC vs CTR)

**Objectifs** :
- Comparer les différents modes opératoires d'AES
- Visualiser les faiblesses d'ECB
- Implémenter CBC et CTR correctement
- Démontrer l'importance des IV/nonces

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os
import secrets
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

## 1. Configuration AES de base

In [ ]:
# Génération de clé AES-128
key = secrets.token_bytes(16)  # 128 bits
print(f"Clé AES (hex): {key.hex()}")
print(f"Taille: {len(key)} bytes = {len(key)*8} bits")

## 2. Mode ECB - DANGEREUX !

**Electronic Codebook** : Chaque bloc est chiffré indépendamment.

⚠️ **JAMAIS utiliser ECB en production !**

In [ ]:
def aes_ecb_encrypt(plaintext: bytes, key: bytes) -> bytes:
    """
    Chiffre avec AES-ECB (MODE INSÉCURISÉ - démo seulement).
    """
    # Padding PKCS7 pour avoir longueur multiple de 16
    padder = padding.PKCS7(128).padder()
    padded = padder.update(plaintext) + padder.finalize()
    
    # Chiffrement ECB
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded) + encryptor.finalize()
    
    return ciphertext

# Test
message = b"ATTACK AT DAWN! " * 3  # Message répétitif
c_ecb = aes_ecb_encrypt(message, key)

print(f"Message: {message}")
print(f"\nCiphertext ECB (hex): {c_ecb.hex()}")
print(f"\n⚠️ Remarquez les répétitions dans le ciphertext !")
print(f"Blocs 1 et 2 identiques: {c_ecb[:16].hex() == c_ecb[16:32].hex()}")

### Démonstration visuelle : ECB sur image

**Exemple célèbre** : Chiffrer une image avec ECB conserve les contours !

In [ ]:
# À COMPLÉTER : Charger image, chiffrer avec ECB, visualiser
# Résultat attendu : On voit encore les contours de l'image originale !

# Exemple de pattern à détecter
pattern = np.array([1,2,3,4] * 100)
print("⚠️ ECB révèle la structure : blocs identiques restent identiques")

## 3. Mode CBC - Cipher Block Chaining

**Principe** : $c_i = E_k(m_i \oplus c_{i-1})$ avec $c_0 = IV$

✅ CPA-sécurisé si IV aléatoire et unique

In [ ]:
def aes_cbc_encrypt(plaintext: bytes, key: bytes) -> tuple[bytes, bytes]:
    """
    Chiffre avec AES-CBC.
    
    Returns:
        (IV, ciphertext)
    """
    # IV aléatoire
    iv = secrets.token_bytes(16)
    
    # Padding
    padder = padding.PKCS7(128).padder()
    padded = padder.update(plaintext) + padder.finalize()
    
    # Chiffrement CBC
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded) + encryptor.finalize()
    
    return iv, ciphertext

def aes_cbc_decrypt(iv: bytes, ciphertext: bytes, key: bytes) -> bytes:
    """
    Déchiffre avec AES-CBC.
    """
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded = decryptor.update(ciphertext) + decryptor.finalize()
    
    # Unpadding
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded) + unpadder.finalize()
    
    return plaintext

# Test
iv, c_cbc = aes_cbc_encrypt(message, key)
decrypted = aes_cbc_decrypt(iv, c_cbc, key)

print(f"IV (hex): {iv.hex()}")
print(f"Ciphertext CBC (hex): {c_cbc.hex()}")
print(f"\nDéchiffrement: {decrypted}")
assert decrypted == message
print("✅ CBC fonctionne correctement")

# Comparaison avec ECB
print(f"\n📊 CBC vs ECB:")
print(f"Blocs CBC identiques: {c_cbc[:16].hex() == c_cbc[16:32].hex()}")
print(f"✅ CBC cache les répétitions !")

### Importance de l'IV

**Expérience** : Réutiliser le même IV révèle des informations !

In [ ]:
# À COMPLÉTER : Chiffrer deux messages avec même IV
# Montrer que le premier bloc révèle m1 ⊕ m2 si IV réutilisé

## 4. Mode CTR - Counter Mode

**Principe** : Transformer block cipher en stream cipher

$c_i = m_i \oplus E_k(\text{nonce} \| i)$

✅ CPA-sécurisé avec nonce unique
✅ Parallélisable
✅ Pas de padding nécessaire

In [ ]:
def aes_ctr_encrypt(plaintext: bytes, key: bytes) -> tuple[bytes, bytes]:
    """
    Chiffre avec AES-CTR.
    
    Returns:
        (nonce, ciphertext)
    """
    # Nonce aléatoire (8 bytes) + compteur (8 bytes) = 128 bits
    nonce = secrets.token_bytes(8)
    
    # CTR mode (pas de padding !)
    cipher = Cipher(
        algorithms.AES(key),
        modes.CTR(nonce + b'\x00' * 8),  # nonce || counter
        backend=default_backend()
    )
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    
    return nonce, ciphertext

def aes_ctr_decrypt(nonce: bytes, ciphertext: bytes, key: bytes) -> bytes:
    """
    Déchiffre avec AES-CTR (identique à l'encryption !).
    """
    cipher = Cipher(
        algorithms.AES(key),
        modes.CTR(nonce + b'\x00' * 8),
        backend=default_backend()
    )
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    
    return plaintext

# Test
nonce, c_ctr = aes_ctr_encrypt(message, key)
decrypted = aes_ctr_decrypt(nonce, c_ctr, key)

print(f"Nonce (hex): {nonce.hex()}")
print(f"Ciphertext CTR (hex): {c_ctr.hex()}")
print(f"\nDéchiffrement: {decrypted}")
assert decrypted == message
print("✅ CTR fonctionne correctement")

# Avantage : pas de padding
print(f"\n📊 Longueurs:")
print(f"Message: {len(message)} bytes")
print(f"Ciphertext CTR: {len(c_ctr)} bytes (identique !)")
print(f"Ciphertext CBC: {len(c_cbc)} bytes (padding ajouté)")

## 5. Comparaison des modes

In [ ]:
import pandas as pd

comparison = pd.DataFrame({
    'Mode': ['ECB', 'CBC', 'CTR'],
    'CPA-Sécurisé': ['❌ NON', '✅ Oui (IV aléatoire)', '✅ Oui (nonce unique)'],
    'Parallélisable': ['✅ Oui', '❌ Non (Déchiffrement: Oui)', '✅ Oui'],
    'Padding': ['Oui', 'Oui', 'Non'],
    'Usage': ['❌ JAMAIS', 'Legacy/TLS 1.2', '✅ Recommandé']
})

print(comparison.to_string(index=False))

## 6. Exercices

### Exercice 1 : Attaque CPA sur ECB
Montrer qu'ECB n'est pas CPA-sécurisé en distinguant les chiffrements de deux messages.

### Exercice 2 : IV prévisible en CBC
Démontrer qu'utiliser $IV = E_k(0)$ (prévisible) rend CBC vulnérable aux attaques CPA.

### Exercice 3 : Nonce reuse en CTR
Montrer la catastrophe de réutiliser un nonce en mode CTR (comme Two-Time Pad).

## Conclusion

**Points clés** :
- ❌ ECB est fondamentalement cassé (blocs identiques révélés)
- ✅ CBC nécessite IV aléatoire et unique
- ✅ CTR est le mode recommandé (rapide, parallélisable, pas de padding)
- ⚠️ IV/nonce management est critique pour la sécurité

**En pratique** : Utiliser des modes AEAD (Authenticated Encryption) comme **AES-GCM** plutôt que chiffrement seul !